<a href="https://colab.research.google.com/github/SanjaghRecommand/Recommend/blob/Sara/test_of_RNN___.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
import pandas as pd
import tensorflow as tf
import torch
import pickle
from tabulate import tabulate

In [ ]:
df1 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/taskfullshare.csv')
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1.head()

,Init,service,user
0,1655018492726,328,1275352
1,1655018824405,92,1275354
2,1655018831538,12,1275355
3,1655018842708,149,1274893
4,1655018856558,42,8604


In [ ]:
df=df1

In [ ]:
mininit= df.Init.min()
df['Time']=df.Init.apply(lambda x: round((x- mininit)/2000000000))
# df.drop(columns='Init', inplace=True)
df= df[df['user'] >5 ]
df.sort_values(by='Init',ascending=True).head(2)

,Init,service,user,Time
40557,1497351631553,0,127,0
40555,1497358176030,0,130,0


In [ ]:
df1['Init'] = pd.to_datetime(df1['Init'],unit='ms')
df1=df1.sort_values(by='Init',ascending=True)
df1.head(2)

,Init,service,user,Time
40557,2017-06-13 11:00:31.553,0,127,0
40555,2017-06-13 12:49:36.030,0,130,0


In [ ]:
df1 = df1.sort_values(by='Init')
df1['date'] = df1['Init'].dt.date
# Extract date and time components
df1['date'] = df1['Init'].dt.date
df1['time'] = df1['Init'].dt.time
df1.head()

,Init,service,user,Time,date,time
40557,2017-06-13 11:00:31.553,0,127,0,2017-06-13,11:00:31.553000
40555,2017-06-13 12:49:36.030,0,130,0,2017-06-13,12:49:36.030000
40554,2017-06-13 13:04:06.644,0,132,0,2017-06-13,13:04:06.644000
40597,2017-06-13 13:24:43.878,0,130,0,2017-06-13,13:24:43.878000
40629,2017-06-14 07:35:55.062,0,134,0,2017-06-14,07:35:55.062000


In [ ]:
df1= df1.drop_duplicates(subset=['date', 'service', 'user'])


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1482958 entries, 40557 to 1681000
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   Init     1482958 non-null  datetime64[ns]
 1   service  1482958 non-null  int64         
 2   user     1482958 non-null  int64         
 3   Time     1482958 non-null  int64         
 4   date     1482958 non-null  object        
 5   time     1482958 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 79.2+ MB


In [ ]:
df1=df1.drop(['date','time'],axis=1)
df1.head(1)

,Init,service,user,Time
40557,2017-06-13 11:00:31.553,0,127,0


In [ ]:
max_len= 10 # len(tf.keras.utils.pad_sequences(seqservice)[0])
item_vocab_size= 588

In [ ]:
seqservice , seqtime ,seqtarget  =[] ,[] ,[] 

for user in df1.user.unique():
  user_data = df1[df1['user'] == user].sort_values('Time')
  seqservice.append(user_data.service.values.tolist()[0:-1])
  seqtime.append(user_data.Time.values.tolist()[0:-1])
  seqtarget.append(user_data.Time.values.tolist()[1:])

  # print(user_data)
  # break
train_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservice, maxlen=max_len), tf.keras.utils.pad_sequences(seqtime, maxlen=max_len), tf.keras.utils.pad_sequences(seqtarget, maxlen=max_len)]))
# save dictionary to person_data.pkl file
with open('/content/drive/MyDrive/Recommendation/train_dict10.pkl', 'wb') as fp:
    pickle.dump(train_dict, fp)
    print('dictionary saved successfully to file')
     

dictionary saved successfully to file


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/train_dict10.pkl', 'rb') as fp:
    # /content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/train_dict10.pkl
    train_dict = pickle.load(fp)

    print(' dictionary loaded')

 dictionary loaded


In [ ]:
#@title Create_train
def create_train_tfdata(train_feat_dict, train_target_tensor,    batch_size, buffer_size=None):
    """
    Create train tf dataset for model train input
    :param train_feat_dict: dict, containing the features tensors for train data
    :param train_target_tensor: np.array(), the training TARGET tensor
    :param batch_size: (int) size of the batch to work with
    :param buffer_size: (int) Optional. Default is None. Size of the buffer
    :return: (tuple) 1st element is the training dataset,
                     2nd is the number of steps per epoch (based on batch size)
    """
    if buffer_size is None:
        buffer_size = batch_size*50

    train_steps_per_epoch = len(train_target_tensor) // batch_size
    train_dataset = tf.data.Dataset.from_tensor_slices((train_feat_dict,train_target_tensor)).cache()
    train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
    train_dataset = train_dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)
    
    return train_dataset, train_steps_per_epoch
  
train_feat_dict = {'item_id': train_dict['item_id'],'nb_days': train_dict['nb_days']}
train_target_tensor = train_dict['target'] 
train_dataset, train_steps_per_epoch = create_train_tfdata(train_feat_dict,  train_target_tensor, batch_size=512)

In [ ]:
#@title BuildModel
def build_model(max_len, item_vocab_size):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],   name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],  name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=16,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=100,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization(
     name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat'
                                   )(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    x = tf.keras.layers.Dense(item_vocab_size*2)(att)
    # x = tf.keras.layers.Dense(32)(x)
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(x) #item_vocab_size

    model = tf.keras.Model(inputs, output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.005),
        loss=loss_function,
        metrics=['sparse_categorical_accuracy'])
    
    return model

In [ ]:


#@title BuildModelHP
def build_modelhp(max_len, item_vocab_size , embedding_item , embedding_nb_days , rnn_units_cat , learning_rate):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                       name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                       name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=embedding_item,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=embedding_nb_days,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization( name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=rnn_units_cat, return_sequences=True,  stateful=False,recurrent_initializer='glorot_normal', name='LSTM_cat')(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(att)

    model = tf.keras.Model(inputs, output)
    model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate),   loss=loss_function, metrics=['sparse_categorical_accuracy'])
    
    return model
     


     


In [ ]:
#@title LossFun
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
#@title FitModel
def fit_model(model, train_dataset, steps_per_epoch, epochs):
    """
    Fit the Keras model on the training dataset for a number of given epochs
    :param model: tf model to be trained
    :param train_dataset: (tf.data.Dataset object) the training dataset
                          used to fit the model
    :param steps_per_epoch: (int) Total number of steps (batches of samples) before 
                            declaring one epoch finished and starting the next epoch.
    :param epochs: (int) the number of epochs for the fitting phase
    :return: tuple (mirrored_model, history) with trained model and model history
    """
        # mirrored_strategy allows to use multi GPUs when available
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
        tf.distribute.experimental.CollectiveCommunication.AUTO)
    
    with mirrored_strategy.scope():
        mirrored_model = model
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

    history = mirrored_model.fit(train_dataset,  steps_per_epoch=steps_per_epoch,epochs=epochs,callbacks=early_stopping, verbose=2)

    return mirrored_model, history
     

In [ ]:
# #@title TrainHP
# from kerastuner.tuners import RandomSearch

# # create a tuner object and define the hyperparameter search space
# tuner = RandomSearch(
#     build_modelhp,
#     objective='val_loss',
#     max_trials=10,
#     directory='my_dir',
#     project_name='my_project',
#     hyperparameters={
#         'max_len': [3,5,10],
#         'item_vocab_size':600,
#         'embedding_item': [16, 32, 64],
#         'embedding_nb_days': [32, 64, 128],
#         'rnn_units_cat': [32, 64, 128],       
#         'learning_rate': [0.2, 0.3, 0.4]
#     })

# # prepare the training and validation data
# ...

# # start the hyperparameter search
# tuner.search(train_dataset, epochs=10)

# # fit_model(model, train_dataset, steps_per_epoch, epochs)
# # print the best hyperparameters found
# best_hyperparams = tuner.get_best_hyperparameters()[0].values
# print("Best Hyperparameters:", best_hyperparams)


In [ ]:
from keras.callbacks import EarlyStopping
#@title Train
hyperparameters={
        'max_len': max_len,
        'item_vocab_size':item_vocab_size,
        'embedding_item': 16,
        'embedding_nb_days': 16,
        'rnn_units_cat': 128,       
        'learning_rate': 0.008    }
model=build_modelhp(**hyperparameters)
fit_model(model, train_dataset,50, 34)     

Epoch 1/34


50/50 - 8s - loss: 0.8905 - sparse_categorical_accuracy: 0.2723 - 8s/epoch - 165ms/step
Epoch 2/34


50/50 - 2s - loss: 0.7146 - sparse_categorical_accuracy: 0.3513 - 2s/epoch - 36ms/step
Epoch 3/34


50/50 - 2s - loss: 0.6646 - sparse_categorical_accuracy: 0.3724 - 2s/epoch - 39ms/step
Epoch 4/34


50/50 - 1s - loss: 0.6465 - sparse_categorical_accuracy: 0.3851 - 1s/epoch - 29ms/step
Epoch 5/34


...

50/50 - 0s - loss: 0.3135 - sparse_categorical_accuracy: 0.4918 - 445ms/epoch - 9ms/step
Epoch 32/34


50/50 - 0s - loss: 0.2546 - sparse_categorical_accuracy: 0.5421 - 444ms/epoch - 9ms/step
Epoch 33/34


50/50 - 1s - loss: 0.1711 - sparse_categorical_accuracy: 0.6438 - 611ms/epoch - 12ms/step
Epoch 34/34


50/50 - 1s - loss: 0.5760 - sparse_categorical_accuracy: 0.4052 - 675ms/epoch - 14ms/step


(<keras.engine.functional.Functional at 0x7fd59c37c3a0>,
 <keras.callbacks.History at 0x7fd59c37f010>)

In [ ]:
servicesdf = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjagh/Recommand/data/servicesdf.csv', index_col='Unnamed: 0')

In [ ]:
def ppredict(userno):
  dftest= df1[df1.user==userno]
  dfuserservice = pd.merge(dftest, servicesdf, on="service")
  dfuserservice.drop(columns='service', inplace= True)
  print(tabulate(dfuserservice, headers='keys', tablefmt='psql'))


  seqservicet , seqtimet ,seqtargett  =[] ,[] ,[] 
  for user in dftest.user.unique():
    user_data = dftest[dftest['user'] == user].sort_values('Time')
    seqservicet.append(user_data.service.values.tolist()[0:-1])
    seqtimet.append(user_data.Time.values.tolist()[0:-1])
    seqtargett.append(user_data.Time.values.tolist()[1:])

  test_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservicet , maxlen=max_len), tf.keras.utils.pad_sequences(seqtimet, maxlen=max_len), tf.keras.utils.pad_sequences(seqtargett, maxlen=max_len)]))
  test_feat_dict = {'item_id': test_dict['item_id'],'nb_days': test_dict['nb_days']}
  y=model.predict(test_feat_dict)
  mmax=0
  mmlist=[]
  for i in range(max_len):
    values= y[0][i].tolist()
    while True:
      if max(values) in mmlist: 
        values[values.index(max(values))]=0
      else:      
        break
    mmax=max(values)
    mmlist.append(mmax)
    print(servicesdf[servicesdf.service==values.index(max(values))].service_title.values)

  finaly= np.zeros((item_vocab_size))
  for i in range(len(y[0])):
      finaly+=y[0][i]
  sortyindex=np.argsort(finaly)[-10:]
  finaly.sort()
  # print(finaly[-10:])
  print('-------------------------------------- Method 2:')
  print( servicesdf[servicesdf.service.isin(sortyindex)].service_title.values)


In [ ]:
ppredict(3345)

+----+----------------------------+--------+--------+-----------------------------------+----------+
|    | Init                       |   user |   Time | service_title                     |   subcat |
|----+----------------------------+--------+--------+-----------------------------------+----------|
|  0 | 2017-12-16 12:30:21.155000 |   3345 |      8 | نصب-آیفون-تصویری                  |        1 |
|  1 | 2021-01-09 11:17:42.393000 |   3345 |     56 | تعمیر-و-تعویض-ال-سی-دی-و-تاچ-گوشی |       71 |
+----+----------------------------+--------+--------+-----------------------------------+----------+
1/1 [==============================] - 0s 23ms/step
['نصب-پوستر-سه-بعدی']
['اجرای-پله-استیل']
['کابل-کشی-داکت-کشی-و-نصب-ترانکینگ']
['نصب-و-تعمیر-موتور-برق']
['ساخت-در-و-پنجره-upvc']
['عیب-یابی-و-تعمیرات-سخت-افزار']
['ایزوگام']
['نصب-راه-اندازی-و-نگهداری-UPS']
['تعمیر-صندلی-اداری']
['نصب-شوفاژ']
-------------------------------------- Method 2:
['نصب-شوفاژ' 'سیم-کشی-ساختمان' 'ساخت-در-و-پنجره-آ

In [ ]:
ppredict(1111)

+----+----------------------------+--------+--------+--------------------------+----------+
|    | Init                       |   user |   Time | service_title            |   subcat |
|----+----------------------------+--------+--------+--------------------------+----------|
|  0 | 2017-09-26 15:14:47.793000 |   1111 |      5 | سرویس-و-تعمیر-کولر-آبی   |        0 |
|  1 | 2017-09-26 15:16:05.766000 |   1111 |      5 | نصب-و-تعمیر-آنتن-دیجیتال |        1 |
|  2 | 2017-09-27 10:29:43.139000 |   1111 |      5 | تعمیر-تلفن               |        1 |
|  3 | 2023-03-14 19:56:49.486000 |   1111 |     91 | هایلایت-و-مش-مو-بانوان   |       68 |
|  4 | 2023-03-16 17:13:24.784000 |   1111 |     91 | پاکسازی-صورت-بانوان      |       38 |
+----+----------------------------+--------+--------+--------------------------+----------+
1/1 [==============================] - 0s 19ms/step
['نصب-پوستر-سه-بعدی']
['اجرای-پله-استیل']
['کابل-کشی-داکت-کشی-و-نصب-ترانکینگ']
['نصب-و-تعمیر-موتور-برق']
['ساخت-در-و-پنج

In [ ]:
ppredict(1514100)

+----+----------------------------+---------+--------+----------------------+----------+
|    | Init                       |    user |   Time | service_title        |   subcat |
|----+----------------------------+---------+--------+----------------------+----------|
|  0 | 2023-04-09 17:54:15.085000 | 1514100 |     92 | نصب-دوربین-مدار-بسته |        1 |
+----+----------------------------+---------+--------+----------------------+----------+
1/1 [==============================] - 0s 28ms/step
['نصب-پوستر-سه-بعدی']
['اجرای-پله-استیل']
['کابل-کشی-داکت-کشی-و-نصب-ترانکینگ']
['نصب-و-تعمیر-موتور-برق']
['ساخت-در-و-پنجره-upvc']
['عیب-یابی-و-تعمیرات-سخت-افزار']
['ایزوگام']
['نصب-راه-اندازی-و-نگهداری-UPS']
['تعمیر-صندلی-اداری']
['ساخت-توری-پنجره']
-------------------------------------- Method 2:
['ساخت-توری-پنجره' 'عیب-یابی-و-تعمیرات-سخت-افزار'
 'کابل-کشی-داکت-کشی-و-نصب-ترانکینگ' 'نصب-راه-اندازی-و-نگهداری-UPS'
 'نصب-و-تعمیر-موتور-برق' 'ساخت-در-و-پنجره-upvc' 'نصب-پوستر-سه-بعدی'
 'ایزوگام' 'تعمیر-ص

In [ ]:
ppredict(60816)

+----+----------------------------+--------+--------+-----------------------+----------+
|    | Init                       |   user |   Time | service_title         |   subcat |
|----+----------------------------+--------+--------+-----------------------+----------|
|  0 | 2018-11-14 09:35:52.006000 |  60816 |     22 | تعمیرات-موبایل-و-تبلت |       71 |
|  1 | 2021-12-27 18:42:54.770000 |  60816 |     72 | تربیت-سگ              |       33 |
|  2 | 2022-05-26 09:01:36.474000 |  60816 |     78 | تعمیر-مبل             |        4 |
+----+----------------------------+--------+--------+-----------------------+----------+
1/1 [==============================] - 0s 20ms/step
['نصب-پوستر-سه-بعدی']
['اجرای-پله-استیل']
['کابل-کشی-داکت-کشی-و-نصب-ترانکینگ']
['نصب-و-تعمیر-موتور-برق']
['ساخت-در-و-پنجره-upvc']
['عیب-یابی-و-تعمیرات-سخت-افزار']
['ایزوگام']
['نصب-راه-اندازی-و-نگهداری-UPS']
['ساخت-در-و-پنجره-آهنی']
['تعمیر-تلفن']
-------------------------------------- Method 2:
['تعمیر-تلفن' 'سیم-کشی-تلفن'

In [ ]:
ppredict(60818)

+----+----------------------------+--------+--------+-----------------------+----------+
|    | Init                       |   user |   Time | service_title         |   subcat |
|----+----------------------------+--------+--------+-----------------------+----------|
|  0 | 2018-11-14 09:42:31.679000 |  60818 |     22 | تعمیرات-موبایل-و-تبلت |       71 |
|  1 | 2018-11-16 12:32:22.530000 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
|  2 | 2018-11-27 15:21:55.131000 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
|  3 | 2018-12-01 09:26:34.923000 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
+----+----------------------------+--------+--------+-----------------------+----------+
1/1 [==============================] - 0s 21ms/step
['نصب-پوستر-سه-بعدی']
['اجرای-پله-استیل']
['کابل-کشی-داکت-کشی-و-نصب-ترانکینگ']
['نصب-و-تعمیر-موتور-برق']
['ساخت-در-و-پنجره-upvc']
['عیب-یابی-و-تعمیرات-سخت-افزار']
['ایزوگام']
['ساخت-در-و-پنجره-آهنی']
['لوله-کشی-فاضلاب']
['لوله-کشی-فاضلا